In [2]:
import tensorflow as tf
from keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import pandas as pd
import numpy as np

from tqdm import tqdm
import yaml

In [3]:
with open("../../config.yaml", "r") as stream:
    try:
        configs = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)

In [4]:
TT = 'train' #(T)rain or (T)est
PATH_TO_EXTRACTED_FEATURES = configs['ROOT_DIR'] + configs['PATH_TO_EXTRACTED_FEATURES']
PATH_TO_TRAIN_DS_PROCESSED = str(configs['ROOT_DIR'] + configs['PATH_TO_TRAIN_DS_PROCESSED'])
PATH_TO_MODELS             = str(configs['ROOT_DIR'] + configs['PATH_TO_MODELS'])
PATH_TO_WEIGHTS            = str(configs['ROOT_DIR'] + configs['PATH_TO_WEIGHTS'])

In [5]:
base = pd.read_csv(PATH_TO_TRAIN_DS_PROCESSED)
base.head()

,image_name,text,humour,sarcasm,offensive,motivational,target
0,image_1415.jpeg,if you want to view paradis simpli look around...,not_funny,not_sarcastic,not_offensive,motivational,positive
1,image_6460.png,if i had a brick for everi lie hillari told i ...,hilarious,general,very_offensive,motivational,positive
2,image_2303.png,that thing over there can i eat that,very_funny,general,not_offensive,motivational,very_positive
3,image_2417.png,my dad point to liam and said when did david b...,not_funny,general,very_offensive,motivational,neutral
4,image_11.jpg,probabl the first man to do year challeng chen...,funny,general,very_offensive,motivational,negative


## EfficientNetV2B0 for feature extraction

In [7]:
model_name = "EfficientNet"

In [3]:
width = 100
height = 100
X_base = []
X_base_path = []
for i in tqdm(base['image_name']):
    path = "dataset/images/"+i
    img = image.load_img(path,target_size=(width,height,3))
    img = image.img_to_array(img)
    X_base.append(img)
    X_base_path.append(path)

100%|██████████| 1369/1369 [00:45<00:00, 30.11it/s]


In [4]:
X_base = np.array(X_base)
Y_base = base.target.astype('category')
Y_train_code = Y_base.cat.codes
n_classes = len(np.unique(base.target.unique()))
Y_base = tf.keras.utils.to_categorical(Y_train_code, n_classes)

datagen_image = ImageDataGenerator(brightness_range=[0.7,1.5],preprocessing_function=tf.keras.applications.resnet50.preprocess_input,channel_shift_range=20)

train_batches = datagen_image.flow(X_base, np.array(Y_base), shuffle=False, batch_size=64)

### Cria o modelo extrator de features

In [5]:
model_EfficientNetV2B0 = tf.keras.models.load_model(PATH_TO_WEIGHTS + 'EfficientNetV2B0_frozen.h5')

for layer in model_EfficientNetV2B0.layers:
    layer.trainable = False

model_img = Model(model_EfficientNetV2B0.input, model_EfficientNetV2B0.layers[-2].output)
feats_img_train = model_img.predict(X_base)

In [6]:
pd.DataFrame(feats_img_train).to_csv(PATH_TO_EXTRACTED_FEATURES + model_name + "-" + TT + ".csv")